# Merge Cleaned Datasets and Deep Cleaning

1. Load cleaned datasets (`IRENA`, `OWID CO₂`, `WGI`)
2. Inspect each dataset for types, missing values, and column consistency
3. Merge datasets step by step using `ISO3` and `year`
4. Handle missing values (fill, drop, or keep as NA)
5. Generate a final combined dataset ready for EDA and visualizations
ISO3 and year alignment across all datasets

Step-by-step merging — beginner-friendly

Missing values handled carefully

Clean final dataset ready for analysis

In [3]:
# Imports and setup
import pandas as pd
from pathlib import Path

clean_dir = Path("../data/cleaned")
final_dir = Path("../data/final")
final_dir.mkdir(exist_ok=True)

In [ ]:
#Load datasets
# Load cleaned IRENA
irena = pd.read_csv(clean_dir / "cleaned_irena.csv")
print("IRENA shape:", irena.shape)
irena.head(2)

# Load cleaned OWID
owid = pd.read_csv(clean_dir / "owid_countries.csv")
print("OWID shape:", owid.shape)
owid.head(2)

# Load cleaned WGI
wgi = pd.read_csv(clean_dir / "wgi_countries.csv")
print("WGI shape:", wgi.shape)
wgi.head(2)

IRENA shape: (91743, 17)
OWID shape: (42480, 80)
WGI shape: (28350, 49)


,codeindyr,code,country,year,indicator,estimate,stddev,nsource,pctrank,pctranklower,...,tpr,vab,vdm,wbs,wcy,wjp,wmo,scalemean,scalesd,country_iso
0,AFGcc1996,AFG,Afghanistan,1996,cc,-1.291704773902893,0.34050697088241577,2,4.301075458526611,0,...,..,..,0.2950838125720781,..,..,..,0,0.013374,0.93648,AFG
1,ALBcc1996,ALB,Albania,1996,cc,-0.8939034938812256,0.3159140348434448,3,19.354839324951172,2.6881721019744873,...,..,..,0.315589909591906,..,..,..,0.25,0.013374,0.93648,ALB


## Check key columns for merging
We will merge datasets on:
- `country_iso` → standardized ISO3 code
- `year` → numeric year

Check if these columns exist and have no missing values

In [ ]:
#list view of column names
print("\nIRENA columns:\n", irena.columns.tolist(),"\n")
print("\nOWID columns:\n", owid.columns.tolist())
print("\nWGI columns:\n", wgi.columns.tolist())

#Column names in separate lines
print("IRENA columns:\n" + "\n".join(irena.columns))
print("\n\nOWID columns:\n" + "\n".join(owid.columns))
print("\n\nWGI columns:\n" + "\n".join(wgi.columns))

In [12]:
# Check IRENA
print(irena[['iso3_code','year']].isna().sum())

# Check OWID
print(owid[['iso_code','year']].isna().sum())

# Check WGI
print(wgi[['country_iso','year']].isna().sum())

iso3_code    0
year         0
dtype: int64
iso_code    0
year        0
dtype: int64
country_iso    0
year           0
dtype: int64


datasets are clean, aligned, and ready to be merged using ISO code + year

In [24]:
## 3. Merge Step-by-Step

# 1. Standardize ISO column names in the 3 datasets
# 2. Merge `IRENA` + `OWID` on `country_iso` and `year` (left join)
# 3. Merge the result with `WGI` (left join)
# 4. Keep all IRENA rows; missing OWID or WGI data will be NaN
# Merge IRENA + OWID

# Make ISO column consistent across datasets
irena = irena.rename(columns={'iso3_code': 'iso'})
owid  = owid.rename(columns={'iso_code': 'iso'})
wgi   = wgi.rename(columns={'country_iso': 'iso'})
# Merge IRENA + OWID
irena_owid = pd.merge(
    irena,
    owid,
    on=['iso','year'],
    how='left',
    suffixes=('_irena','_owid')
)
print("IRENA + OWID shape:", irena_owid.shape)

# Merge with WGI
final_df = pd.merge(
    irena_owid,
    wgi,
    on=['iso','year'],
    how='left',
    suffixes=('','_wgi')
)
print("Final merged shape:", final_df.shape)
#Save final dataset
final_df.to_csv("../data/final/final_countries.csv", index=False)
final_df.head(3)

IRENA + OWID shape: (91743, 95)
Final merged shape: (472473, 142)


,region,sub-region,country_irena,iso,m49_code,re_or_non-re,group_technology,technology,sub-technology,producer_type,...,rsf,tpr,vab,vdm,wbs,wcy,wjp,wmo,scalemean,scalesd
0,Africa,Northern Africa,Algeria,DZA,12,Total Renewable,Bioenergy,Solid biofuels,Other primary solid biofuels n.e.s.,All types,...,..,..,..,0.297942187677093,..,..,..,0.25,0.004617,0.932170
1,Africa,Northern Africa,Algeria,DZA,12,Total Renewable,Bioenergy,Solid biofuels,Other primary solid biofuels n.e.s.,All types,...,..,..,..,..,..,..,..,0.25,-0.036084,0.972304
2,Africa,Northern Africa,Algeria,DZA,12,Total Renewable,Bioenergy,Solid biofuels,Other primary solid biofuels n.e.s.,All types,...,..,..,..,..,..,..,..,0.25,0.018476,0.935971


In [63]:
## 4. Check Data Types
# Show all columns without truncation
# pd.set_option('display.max_columns', None)
# final_df.dtypes
print(final_df.dtypes.to_frame('dtype').to_string())

                                                        dtype
region                                               category
sub-region                                           category
country_irena                                        category
iso                                                  category
m49_code                                                int64
re_or_non-re                                         category
group_technology                                     category
technology                                           category
sub-technology                                       category
producer_type                                        category
year                                           datetime64[ns]
electricity_generation_(gwh)                          float64
electricity_installed_capacity_(mw)                   float64
heat_generation_(tj)                                  float64
public_flows_(2022_usd_m)                             float64
sdg_7a1_

Type Conversion Summary
1.Many numeric columns (e.g., population, gdp, co2) are stored as object → need conversion to numeric for analysis.
2.year is currently int64 → can remain as is or converted to datetime for time-series operations.
3.Columns like region, country names, and ISO codes are object → can be converted to category to save memory.

Actions Taken
1.Convert numeric columns stored as object → float64/int64.
2.Convert year →  convert to datetime.
3.Convert categorical columns (region, iso, country names, etc.) → category.

Benefits
1.Enables proper numeric calculations and aggregations.
2.Optimizes memory usage.
3.Prepares dataset for time-series and categorical analyses.

In [64]:
# 1.Convert numeric columns stored as object to numeric
# List numeric columns that may be stored as object
numeric_cols = [
    'population', 'gdp', 'cement_co2', 'co2', 'co2_per_capita', 'coal_co2', 'oil_co2',
    'gas_co2', 'methane', 'nitrous_oxide', 'primary_energy_consumption',
    'electricity_generation_(gwh)', 'electricity_installed_capacity_(mw)',
    'heat_generation_(tj)', 'public_flows_(2022_usd_m)',
    'sdg_7a1_intl._public_flows_(2022_usd_m)',
    'sdg_7b1_re_capacity_per_capita_(w/inhabitant)',
    'temperature_change_from_co2', 'temperature_change_from_ch4', 
    'temperature_change_from_ghg', 'total_ghg', 'total_ghg_excluding_lucf',
    'scalemean', 'scalesd'
]

# 2.Convert to numeric, coercing errors to NaN
for col in numeric_cols:
    if col in final_df.columns:
        final_df[col] = pd.to_numeric(final_df[col], errors='coerce')
        

#Convert year column to datetime
# Keep as int64
#final_df['year'] = final_df['year'].astype('int64')

# Or convert to datetime for time-series operations
final_df['year'] = pd.to_datetime(final_df['year'], format='%Y')

#3.Convert ISO codes, country names, and categorical labels to category
cat_cols = [
    'region', 'sub-region', 'country_irena', 'iso', 're_or_non-re', 
    'group_technology', 'technology', 'sub-technology', 'producer_type',
    'country_owid', 'country_iso'
]

for col in cat_cols:
    if col in final_df.columns:
        final_df[col] = final_df[col].astype('category')

print(final_df.dtypes.to_frame('dtype').to_string())

                                                        dtype
region                                               category
sub-region                                           category
country_irena                                        category
iso                                                  category
m49_code                                                int64
re_or_non-re                                         category
group_technology                                     category
technology                                           category
sub-technology                                       category
producer_type                                        category
year                                           datetime64[ns]
electricity_generation_(gwh)                          float64
electricity_installed_capacity_(mw)                   float64
heat_generation_(tj)                                  float64
public_flows_(2022_usd_m)                             float64
sdg_7a1_

## 5. Check Missing Values

- Some countries or years may be missing OWID or WGI data
- Decide on imputation or leave as NA
- For numeric columns we can fill with 0 or median (depends on context)

In [25]:
# Missing values count
missing_counts = final_df.isna().sum().sort_values(ascending=False)
missing_counts.head(20)

heat_generation_(tj)                             408790
sdg_7b1_re_capacity_per_capita_(w/inhabitant)    312711
cumulative_other_co2                             283751
other_industry_co2                               283751
other_co2_per_capita                             283751
share_global_other_co2                           283751
share_global_cumulative_other_co2                283751
electricity_installed_capacity_(mw)              276116
electricity_generation_(gwh)                     274344
consumption_co2_per_gdp                          148267
consumption_co2                                  129787
consumption_co2_per_capita                       129787
trade_co2_share                                  129787
trade_co2                                        129787
gas_co2_per_capita                               127896
gas_co2                                          127896
share_global_cumulative_gas_co2                  127896
share_global_gas_co2                            

In [ ]:
#Analysing heat_generation_(tj) column
final_df['heat_generation_(tj)'].describe()

missing_by_country = final_df[final_df['heat_generation_(tj)'].isna()].groupby('iso').size().sort_values(ascending=False)
print(missing_by_country)

# Calculate median heat generation per country
median_by_country = final_df.groupby('iso')['heat_generation_(tj)'].median()

# View top 10
print(median_by_country.sort_values(ascending=False).head(10))
from scipy.stats import skew

# Select the column
heat = final_df['heat_generation_(tj)']

# Drop missing values temporarily for analysis
heat_nonan = heat.dropna()

# Calculate basic stats
mean_val = heat_nonan.mean()
median_val = heat_nonan.median()
std_val = heat_nonan.std()
skewness = skew(heat_nonan)

print(f"Mean: {mean_val:.2f}")
print(f"Median: {median_val:.2f}")
print(f"Std Dev: {std_val:.2f}")
print(f"Skewness: {skewness:.2f}")

# Group by country
country_stats = final_df.groupby('iso')['heat_generation_(tj)'].agg(
    count='count',
    missing='size',
    mean='mean',
    median='median',
    std='std',
    skew=lambda x: skew(x.dropna())
).reset_index()

# Calculate missing values per country
country_stats['missing'] = final_df.groupby('iso')['heat_generation_(tj)'].apply(lambda x: x.isna().sum()).values

print(country_stats.head(10))


iso
BRA    4392
ARG    4301
ESP    3617
THA    3592
IND    3542
       ... 
XAA      54
XOC      36
XMX      15
EUE       7
SHN       5
Length: 183, dtype: int64
iso
CHN    114078.500
RUS    113722.000
KAZ     62583.000
KGZ     13020.529
USA     12224.000
JPN      7882.000
DEU      6527.000
FRA      6493.000
UKR      5879.000
UZB      4737.000
Name: heat_generation_(tj), dtype: float64
Mean: 15665.55
Median: 1366.00
Std Dev: 124824.93
Skewness: 24.52
   iso  count  missing        mean  median         std      skew
0  ABW      0     1223         NaN     NaN         NaN       NaN
1  AFG      0     1958         NaN     NaN         NaN       NaN
2  AGO      0     1710         NaN     NaN         NaN       NaN
3  AIA      0     1053         NaN     NaN         NaN       NaN
4  ALB   1510        0  123.500000   123.5    5.784425  2.385929
5  AND      0     1205         NaN     NaN         NaN       NaN
6  ARE      0     1590         NaN     NaN         NaN       NaN
7  ARG      0     4301   

C:\Users\Code Lou\AppData\Local\Temp\ipykernel_8108\3670370555.py:37: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  skew=lambda x: skew(x.dropna())


In [44]:
# Step 0: Make sure column name is clean
final_df.columns = final_df.columns.str.strip()

# Step 1: Check missing before
print("Missing before:", final_df['heat_generation_(tj)'].isna().sum())

# Step 2: Impute per country; if all NaN, keep as NaN
final_df['heat_generation_(tj)'] = final_df.groupby('iso')['heat_generation_(tj)'].transform(
    lambda x: x.fillna(x.median()) if not x.isna().all() else x
)

# Step 3: Check missing after
print("Missing after:", final_df['heat_generation_(tj)'].isna().sum())

# View top 10
print(median_by_country.sort_values(ascending=False))


Missing before: 0
Missing after: 0
iso
CHN    114078.500
RUS    113722.000
KAZ     62583.000
KGZ     13020.529
USA     12224.000
          ...    
XOC           NaN
YEM           NaN
ZAF           NaN
ZMB           NaN
ZWE           NaN
Name: heat_generation_(tj), Length: 235, dtype: float64


Handling Missing Values in Energy & Socioeconomic Data
1.Energy Generation Columns
heat_generation_(TJ), electricity_generation_(GWh), electricity_installed_capacity_(MW)
Fill missing values per country using median. If all values for a country are missing, leave as NaN.

2.Population & GDP Columns
population, gdp
Fill missing values using interpolation along time for the same country. Avoid filling with 0 to prevent distortion in per-capita metrics.

3.Derived / Aggregate Columns
co2_per_capita, sdg_7b1_re_capacity_per_capita, trade_co2_share, etc.
Recalculate after filling the raw data, or fill missing using country-wise median if necessary.

In [46]:
#Code Imputation
#Part 1: Energy Generation Columns
# Ensure column names are clean
final_df.columns = final_df.columns.str.strip()

energy_cols = [
    'heat_generation_(tj)',
    'electricity_generation_(gwh)',
    'electricity_installed_capacity_(mw)'
]

# Impute missing values per country using median
for col in energy_cols:
    if col in final_df.columns:
        final_df[col] = final_df.groupby('iso')[col].transform(
            lambda x: x.fillna(x.median()) if not x.isna().all() else x
        )

# Check missing after
print("Missing values after imputing energy columns:")
print(final_df[energy_cols].isna().sum())

#Part 2: Population & GDP Columns
pop_gdp_cols = ['population', 'gdp']

# Interpolate missing values along time per country using transform
for col in pop_gdp_cols:
    if col in final_df.columns:
        final_df[col] = final_df.groupby('iso')[col].transform(
            lambda x: x.interpolate(method='linear', limit_direction='both')
        )

# Check missing after
print("Missing values after imputing population & GDP:")
print(final_df[pop_gdp_cols].isna().sum())

Missing values after imputing energy columns:
heat_generation_(tj)                     0
electricity_generation_(gwh)           753
electricity_installed_capacity_(mw)    753
dtype: int64
Missing values after imputing population & GDP:
population    13149
gdp           61438
dtype: int64


In [60]:
# Part 3: Derived / Aggregate Columns
# ==============================
derived_cols = [
    'co2_per_capita', 
    'sdg_7b1_re_capacity_per_capita_(w/inhabitant)',
    'trade_co2_share',
    'cumulative_other_co2'
]

for col in derived_cols:
    if col in final_df.columns:
        # Option 1: Recalculate if raw data available
        # Option 2: Impute per country median if all values not missing
        final_df[col] = final_df.groupby('iso')[col].transform(
            lambda x: x.fillna(x.median()) if not x.isna().all() else x
        )

print("Missing values after imputing derived columns:")
print(final_df[derived_cols].isna().sum())


Missing values after imputing derived columns:
co2_per_capita                                    13149
sdg_7b1_re_capacity_per_capita_(w/inhabitant)       821
trade_co2_share                                  127800
cumulative_other_co2                             283751
dtype: int64


In [61]:
## 6. Quick Descriptive Stats
final_df.describe(include='all').T.head(10)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
region,472473,7,Europe,167589,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-region,472473,21,Sub-Saharan Africa,79546,NaN,NaN,NaN,NaN,NaN,NaN,NaN
country_irena,472473,235,Sweden,7338,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iso,472473,235,SWE,7338,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m49_code,472473.0,NaN,NaN,NaN,579.092564,3929.935867,4.0,208.0,417.0,642.0,99909.0
re_or_non-re,472473,2,Total Renewable,347471,NaN,NaN,NaN,NaN,NaN,NaN,NaN
group_technology,472473,12,Bioenergy,108277,NaN,NaN,NaN,NaN,NaN,NaN,NaN
technology,472473,23,Solid biofuels,66864,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub-technology,472473,47,Renewable hydropower,54625,NaN,NaN,NaN,NaN,NaN,NaN,NaN
producer_type,472473,5,All types,191106,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
## 6. Save final merged dataset
final_path = final_dir / "final_combined.csv"
final_df.to_csv(final_path, index=False)
print("Saved final dataset:", final_path)
